`Lab creado por Margarita Geleta para el curso Introducción a Machine Learning JEDI, edición 2021`

# [III] Optimización

> **Derivar, es inevitable.** ¿Te acuerdas cómo se deriva una función?

La optimización es clave en técnicas de Machine Learning. Cuando definimos un modelo $f$ para aprender un patrón, también definimos una función de coste $J$ (una función de error). Esta función va definida por nuestros datos $x_1, x_2, ..., x_N$ que son fijos igual que las etiquetas $y_1, y_2, ..., y_N$. El modelo $f$ recibe como parámetero el dato $x_k$ y un parámetro $w$ (o una série de parámetros), y debe predecir, idealmente, $y_k$. La función de error calcula cuánto nos equivocamos, cuánto se desvia la predicción $f(x_k, w)$ de la realidad $y_k$.

Comencemos por algo fácil. Supongamos que nuestra función de error es tan simple como la resta de $f(x_k, w)$ y $y_k$ al cuadrado (para que sea positivo):

$J(x_k, w) = (f(x_k, w) - y_k)^2$ --> esto es el error para un punto en concreto

$J(w) = \sum_{k = 1}^N J(x_k, w) = \sum_{k = 1}^N  (f(x_k, w) - y_k)^2$ --> el error del modelo es la suma de todos los errores puntuales

Esta función de error se conoce por el nombre de **Error Cuadrático Medio** (https://en.wikipedia.org/wiki/Mean_squared_error).

In [1]:
import numpy as np
import matplotlib.pyplot as plt

N = 200
X = np.random.rand(N) * 10
secreto = 2 * X - 4



ModuleNotFoundError: No module named 'numpy'

Acabamos de crear 200 puntos $x_1, x_2, ..., x_{200}$. La función $2 * X - 4$ es la función que queremos modelar con Machine Learning y tiene dos parámetros $w$: $w_1 = 2$ y $w_2 = -4$. Estos son los parámetros que intentaremos determinar con el modelo. 

Recibimos las etiquetas con cierto error:

In [ ]:
error = np.random.randn(N) * 2.25 # error de la distribución normal
Y = 2 * X - 4 + error

In [ ]:
plt.scatter(X, Y, label = 'Datos reales')
plt.plot(X, secreto, label = 'La función a modelar')
plt.legend()

Definamos la función de error ... ¿Cómo se define un sumatorio en Python?

$J(w) = \sum_{k = 1}^N  (f(x_k, w) - y_k)^2$

$f(x_k, w) = x_k \cdot w_1 + w_2$

In [4]:
def J(w):
    return sum((w[0]*X[k] + w[1] - Y[k])**2 for k in range(0, N))

El mínimo error que se puede cometer es pues ...

In [ ]:
J([2, -4]) # los parámetros OPTIMOS

Fíjate que al variar un poco los parámetros, el error incrementa:

In [ ]:
J([2.01, -3.8])

Al comenzar el entrenamiento del modelo $f$ no tenemos ni idea de los valores $w_1$ y $w_2$. Por eso, escogeríamos unos valores al azar y a partir de ahí intentaremos optimizar para llegar al valor óptimo de la función de coste $J(w)$.  

In [ ]:
w = np.array([np.random.rand(2)]).T # Escojamos 2 valores aleatorios
w

Sabemos que no nos encontramos en el óptimo ($w_1 = 2, w_2 = -4$). ¿Cómo podemos bajar?

<div style="display:flex; justify-content: center;">
    <img src="data/convex1.png" alt="drawing" style="height:500px;"/>
    <img src="data/convex2.png" alt="drawing" style="height:500px;"/>
</div>

In [ ]:
plt.scatter(X, Y, label = 'Datos reales')
plt.plot(X, secreto, label = 'La función a modelar')
plt.plot(X, w[0]*X + X[1], label = 'Inicialización aleatoria')
plt.legend()

Para descender de un punto a un mínimo en una función existen algoritmos de optimización que utilizan diferentes estrategias. El algoritmo de optimización más básico es el **Gradiente Descendiente** (https://en.wikipedia.org/wiki/Gradient_descent). La idea es la siguiente:

- El gradiente de una función es la derivada de una función. El gradiente siempre apunta a la dirección de máximo crecimiento. 
- La dirección contraria llevaría a la dirección de decrecimiento.
- Intentaremos bajar por la función de error por la dirección negativa del gradiente.

Para aplicar este algoritmo necesitamos calcular antes la derivada de la función de coste:

$J(w) = \sum_{k = 1}^N  (x_k \cdot w_1 + w_2 - y_k)^2$

$\frac{\partial f(x_k, w)}{\partial w_1} = 1/N \cdot \sum_{k = 1}^N  2\cdot x_k \cdot (x_k \cdot w_1 + w_2 - y_k)$

$\frac{\partial f(x_k, w)}{\partial w_2} = 1/N \cdot \sum_{k = 1}^N  2\cdot (x_k \cdot w_1 + w_2 - y_k)$

$\frac{\partial J(w)}{\partial w} = 2/N\cdot \left(\sum_{k = 1}^N  x_k \cdot (x_k \cdot w_1 + w_2 - y_k) ,  \sum_{k = 1}^N  (x_k \cdot w_1 + w_2 - y_k) \right)$

In [1]:
def dJ(w):
    gradient_w0 = (2 / N) * sum(X[k]*(w[0]*X[k] + w[1] - Y[k]) for k in range(0, N))
    gradient_w1 = (2 / N) * sum((w[0]*X[k] + w[1] - Y[k]) for k in range(0, N))
    return np.array([gradient_w0, gradient_w1])

In [2]:
dJ([2, -4])

NameError: name 'N' is not defined

Descargamos la líbreria con algoritmos de optimización:

In [ ]:
! pip3 install -i https://test.pypi.org/simple/ optrita==0.0.3
"""
Si no va, probar:
import sys
!{sys.executable} -m pip install -i https://test.pypi.org/simple/ optrita==0.0.3
"""
import optrita as opt

Al algoritmo le pasamos la siguiente información:
- El punto inicial $w$.
- La función de error $J$.
- La derivada de la función de error $\frac{\partial J(w)}{\partial w}$.
- Parámetros de optimización.

In [ ]:
BLS_params = dict(alpha_max = 2.0,
                  alpha_min = 1e-3,
                  rho = 0.5,
                  c1 = 0.01, c2 = 0.45,
                  strong_wolfe = True, out = 0)

Xk_GM, info_GM = opt.GM(w, J, dJ, BLS_params, eps = 1e-6, kmax = 1500, precision = 6)

In [ ]:
Xk_GM

Como que la función de coste está definida solamente por dos parámetros, la función de coste se puede visualizar en un plot tridimensional. Podemos visualizar el descenso:

In [ ]:
opt.contour_map(J, Xk_GM[0], Xk_GM[1])

También podemos observar con qué rapidez disminuye la norma del gradiente (cómo más próxima sea a 0, más cerca estamos a un punto crítico). A veces es útil verlo en escala logarítmica.

In [ ]:
plt.plot(info_GM['||g(x)||'])
#plt.yscale('log')

También podemos ver los diferentes modelos que se han ido generando:

In [ ]:
plt.scatter(X, Y, label = 'Datos reales')
for i in range(0, len(Xk_GM[0]), 10):
    plt.plot(X, Xk_GM[0][i]*X + Xk_GM[1][i], label = i)

In [ ]:
plt.scatter(X, Y, label = 'Datos reales')
plt.plot(X, secreto, label = 'La función a modelar')
plt.plot(X, Xk_GM[0][-1]*X + Xk_GM[1][-1], label = 'Nuestro modelo')
plt.legend()

Esta es la base de Machine Learning:
- Se define una función de coste (función de error) según el problema.
- Se escoge un algoritmo de optimización.
- Se optimiza con el algoritmo en la función de coste para encontrar los parámetros óptimos.

Cuidado!
- No hay que optimizar demasiado porque sino podemos hacer overfitting.
- Optimizando poco, hacemos underfitting.
- Hemos visto una función de coste muy simple. En la vida real, tenemos muchos paráemtros $w$, las funciones no son bidimensionales, tienen muchas dimensiones, son muy complejas, tienen muchos puntos críticos (aparte de mínimos locales, también puntos de silla), no lo podemos visualizar y es más difícil optimizar.

¿Qué buscamos?
- Buscamos algoritmos de optimización de vayan muy rápidos (--> convergencia local).
- Buscamos algoritmos de optimización que lleguen a un mínimo local (--> convergencia global).

GM (Gradient Descent) es el algoritmo de optimización más básico. CGM (Conjugate Gradient Descent) aparte del gradiente de la iteración actual, también utiliza el gradiente de la iteración anterior y la dirección de descenso se calcula a partir de una combinación lineal de gradientes. Vamos a probar - Hay dos variaciones de CGM:

In [ ]:
Xk_CGM, info_CGM = opt.CGM(w, J, dJ, BLS_params, iCG = 1, iRC = 1, eps = 1e-6, kmax = 1500, precision = 6,nu = 0.1)

In [ ]:
opt.contour_map(J, Xk_CGM[0], Xk_CGM[1])

In [ ]:
Xk_CGM, info_CGM = opt.CGM(w, J, dJ, BLS_params, iCG = 2, iRC = 1, eps = 1e-6, kmax = 1500, precision = 6,nu = 0.1)

In [ ]:
opt.contour_map(J, Xk_CGM[0], Xk_CGM[1])

Mucho más rápido!

BFGS (Broyden-Fletcher-Goldfarb-Shanno)

In [ ]:
Xk_BFGS, info_BFGS = opt.BFGS(w, J, dJ, BLS_params, eps = 1e-6, kmax = 1500, precision = 6)

In [ ]:
opt.contour_map(J, Xk_BFGS[0], Xk_BFGS[1])

In [ ]:
plt.plot(info_GM['||g(x)||'], label = 'GM')
plt.plot(info_CGM['||g(x)||'], label = 'CGM')
plt.plot(info_BFGS['||g(x)||'], label = 'BFGS')
plt.xscale('log')
plt.yscale('log')
plt.legend()

Hasta ahora solo hemos utilizado información de la primera derivada (el gradiente).
También hay algoritmos de optimización que utilizan información de la segunda derivada (la Hessiana), que aporta información sobre la curvatura de la función y ayuda a encontrar el camino de descenso más rápido. El inconveniente de estos algoritmos es que se tiene que calcular la Hessiana (derivada de derivada).

Si te interesa la implementación de estos algoritmos de optimización, la puedes encontrar en mi GitHub:
https://github.com/margaritageleta/optrita/blob/master/optrita/linesearch/base.py

## Vamos a experimentar con los algoritmos de optimización 

### Ejemplo 1

$f(x,y) = x^2 + y^2$

<div style="display:flex; justify-content: center; align-items: center;">
    <img src="data/convex.png" alt="drawing" style="height:500px;"/>
</div>

Definamos la función, su gradiente y un punto inicial:

In [ ]:
# Problem
def f(x): return x[0]**2 + x[1]**2 
def g(x): return np.array([2*x[0], 2*x[1]])

x = np.array([[50, -50]]).T

BLS_params = dict(alpha_max = 2.0,
                  alpha_min = 1e-3,
                  rho = 0.5,
                  c1 = 0.01, c2 = 0.45,
                  strong_wolfe = True, out = 0)

Xk_GM, info_GM = opt.GM(x, f, g, BLS_params, eps = 1e-6, kmax = 1500, precision = 6)
Xk_CGM, info_CGM = opt.CGM(x, f, g, BLS_params, iCG = 1, iRC = 1, eps = 1e-6, kmax = 1500, precision = 6,nu = 0.1)
Xk_BFGS, info_BFGS = opt.BFGS(x, f, g, BLS_params, eps = 1e-6, kmax = 1500, precision = 6)

In [ ]:
opt.contour_map(f, Xk_GM[0], Xk_GM[1])

La función es muy simple, los algoritmos encuentran el óptimo en una iteración. Hasta ahora hemos hablado sobre cómo se escoge la dirección del descenso (el gradiente negativo, o la combinación de gradientes, etc.) pero no hemos hablado sobre la "longitud" de paso en el momento de bajar. Es aquí donde entra en juego el paquete de parámetros de optimización `BLS_params`. Más adelante, vamos a ver funciones de error más complicadas y el efecto que tienen estos parámetros en la optimización.

### Ejemplo 2

$f(x,y) = -x^3 - \frac{1}{2}x^2 + \frac{1}{2}y^2 + \frac{1}{4}xy + x$

In [ ]:
# Problem
def f(x): return -x[0]**3 - (1/2)*x[0]**2 + (1/2)*x[1]**2 + (1/4)*x[0]*x[1] + x[0]
def g(x): return np.array([-3*x[0]**2 - x[0] + (1/4)*x[1] + 1,
                           x[1] + (1/4)*x[0]])

x = np.array([[0.4, 0.6]]).T

BLS_params = dict(alpha_max = 2.0,
                  alpha_min = 1e-3,
                  rho = 0.5,
                  c1 = 0.01, c2 = 0.45,
                  strong_wolfe = True, out = 0)

Xk_GM, info_GM = opt.GM(x, f, g, BLS_params, eps = 1e-6, kmax = 1500, precision = 6)
Xk_CGM, info_CGM = opt.CGM(x, f, g, BLS_params, iCG = 2, iRC = 1, eps = 1e-6, kmax = 1500, precision = 6,nu = 0.1)
Xk_BFGS, info_BFGS = opt.BFGS(x, f, g, BLS_params, eps = 1e-6, kmax = 1500, precision = 6)

In [ ]:
plt.plot(info_GM["||g(x)||"], label = "GM")
plt.plot(info_CGM["||g(x)||"], label = "CGM")
plt.plot(info_BFGS["||g(x)||"], label = "BFGS")
plt.legend()
plt.show()

In [ ]:
opt.contour_map(f, Xk_GM[0], Xk_GM[1])

In [ ]:
opt.contour_map(f, Xk_CGM[0], Xk_CGM[1])

In [ ]:
opt.contour_map(f, Xk_BFGS[0], Xk_BFGS[1])

Dependiendo del punto inicial, a veces encontramos soluciones diferentes ...
Hay que ir con cuidado, a veces podemos caer a puntos de silla!

In [ ]:
x = np.array([[0.1, 0.6]]).T
Xk_GM, info_GM = opt.GM(x, f, g, BLS_params, eps = 1e-6, kmax = 1500, precision = 6)
Xk_BFGS, info_BFGS = opt.BFGS(x, f, g, BLS_params, eps = 1e-6, kmax = 1500, precision = 6)

In [ ]:
opt.contour_map(f, Xk_GM[0], Xk_GM[1])

In [ ]:
opt.contour_map(f, Xk_BFGS[0], Xk_BFGS[1])

### Ejemplo 3

Aquí tenemos un ejemplo de una función con 3 paráemtros (ya no la podemos visualizar, es 4D):

$f(x_0,x_1,x_2) = (x_0 - 1)^2 + x_1^3 + x_2^2$

$\frac{\partial f}{\partial x_0} = 2(x_0 - 1) \cdot 1$
,$\frac{\partial f}{\partial x_1} = 3x_1^2$
,$\frac{\partial f}{\partial x_2} = 2x_2$

In [ ]:
# Problem
def f(x): return (x[0] - 1)**2 + x[1]**3 + x[2]**2
def g(x): return np.array([2*(x[0] - 1),
                           3*x[1]**2,
                           2*x[2]])

x = np.array([[0.8, 0.1, 0.3]]).T

BLS_params = dict(alpha_max = 2.0,
                  alpha_min = 1e-3,
                  rho = 0.4,
                  c1 = 0.01, c2 = 0.45,
                  strong_wolfe = True, out = 0)
Xk_GM, info_GM = opt.GM(x, f, g, BLS_params, eps = 1e-6, kmax = 1200, precision = 6)
Xk_CGM, info_CGM = opt.CGM(x, f, g, BLS_params, iCG = 2, iRC = 1,  eps = 1e-6, kmax = 1200, precision = 6, nu = 9)
Xk_BFGS, info_BFGS = opt.BFGS(x, f, g, BLS_params, eps = 1e-6, kmax = 1200, precision = 6)

In [ ]:
plt.plot(info_GM["||g(x)||"], label = "GM")
plt.plot(info_CGM["||g(x)||"], label = "CGM")
plt.plot(info_BFGS["||g(x)||"], label = "BFGS")
#plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.show()

### Más ejemplos

BFGS ha demostrado ser mucho más rápido que CGM y GM en muchos casos. Pero no siempre es así.
No hay que menospreciar GM, aunque sea el más simple de todos.

In [ ]:
def f(x): return x[0]*np.exp(-x[0]**2-x[1]**2)
def g(x): return np.array([np.exp(-x[0]**2-x[1]**2)*(1 - 2*x[0]**2),
                           np.exp(-x[0]**2-x[1]**2)*(-2*x[0]*x[1])])
x = np.array([[-1,1]]).T

BLS_params = dict(alpha_max = 2.0,
                  alpha_min = 1e-3,
                  rho = 0.4,
                  c1 = 0.01, c2 = 0.45,
                  strong_wolfe = True, out = 0)

Xk_GM, info_GM = opt.GM(x, f, g, BLS_params, eps = 1e-6, kmax = 1500, precision = 6)
Xk_CGM, info_CGM = opt.CGM(x, f, g, BLS_params, iCG = 1, iRC = 1, eps = 1e-6, kmax = 1500, precision = 6,nu = 0.1)
Xk_BFGS, info_BFGS = opt.BFGS(x, f, g, BLS_params, eps = 1e-6, kmax = 1500, precision = 6)

In [ ]:
opt.contour_map(f, Xk_GM[0], Xk_GM[1])

In [ ]:
opt.contour_map(f, Xk_CGM[0], Xk_CGM[1])

In [ ]:
opt.contour_map(f, Xk_BFGS[0], Xk_BFGS[1])

In [ ]:
BLS_params = dict(alpha_max = 2.0,
                  alpha_min = 1e-3,
                  rho = 0.9,
                  c1 = 0.1, c2 = 0.8,
                  strong_wolfe = True, out = 0)
Xk_CGM, info_CGM = opt.CGM(x, f, g, BLS_params, iCG = 1, iRC = 1, eps = 1e-6, kmax = 1500, precision = 6,nu = 0.1)
opt.contour_map(f, Xk_CGM[0], Xk_CGM[1])

In [ ]:
BLS_params = dict(alpha_max = 2.0,
                  alpha_min = 1e-3,
                  rho = 0.2,
                  c1 = 0.1, c2 = 0.8,
                  strong_wolfe = True, out = 0)
Xk_CGM, info_CGM = opt.CGM(x, f, g, BLS_params, iCG = 1, iRC = 1, eps = 1e-6, kmax = 1500, precision = 6,nu = 0.1)
opt.contour_map(f, Xk_CGM[0], Xk_CGM[1])

<div style="display:flex; justify-content: center; align-items: center;">
    <img src="data/convex3.png" alt="drawing" style="height:200px;"/>
</div>

In [ ]:
BLS_params = dict(alpha_max = 2.0,
                  alpha_min = 1e-3,
                  rho = 0.9,
                  c1 = 0.5, c2 = 0.8,
                  strong_wolfe = True, out = 0)
Xk_CGM, info_CGM = opt.CGM(x, f, g, BLS_params, iCG = 1, iRC = 1, eps = 1e-6, kmax = 1500, precision = 6,nu = 0.1)
opt.contour_map(f, Xk_CGM[0], Xk_CGM[1])

In [ ]:
BLS_params = dict(alpha_max = 2.0,
                  alpha_min = 1e-3,
                  rho = 0.2,
                  c1 = 0.01, c2 = 0.01,
                  strong_wolfe = True, out = 0)
Xk_CGM, info_CGM = opt.CGM(x, f, g, BLS_params, iCG = 1, iRC = 1, eps = 1e-6, kmax = 1500, precision = 6,nu = 0.1)
opt.contour_map(f, Xk_CGM[0], Xk_CGM[1])

In [ ]:
BLS_params = dict(alpha_max = 2.0,
                  alpha_min = 1e-3,
                  rho = 0.01,
                  c1 = 0.1, c2 = 0.99,
                  strong_wolfe = True, out = 0)
Xk_CGM, info_CGM = opt.CGM(x, f, g, BLS_params, iCG = 1, iRC = 1, eps = 1e-6, kmax = 1500, precision = 6,nu = 0.1)
opt.contour_map(f, Xk_CGM[0], Xk_CGM[1])

In [ ]:
BLS_params = dict(alpha_max = 7.0,
                  alpha_min = 1e-3,
                  rho = 0.9,
                  c1 = 0.1, c2 = 0.8,
                  strong_wolfe = True, out = 0)
Xk_CGM, info_CGM = opt.CGM(x, f, g, BLS_params, iCG = 1, iRC = 1, eps = 1e-6, kmax = 1500, precision = 6,nu = 0.1)
opt.contour_map(f, Xk_CGM[0], Xk_CGM[1])

In [ ]:
BLS_params = dict(alpha_max = 2.0,
                  alpha_min = 1e-3,
                  rho = 0.9,
                  c1 = 0.1, c2 = 0.8,
                  strong_wolfe = True, out = 0)
Xk_CGM, info_CGM = opt.CGM(x, f, g, BLS_params, iCG = 2, # modificar
                           iRC = 1, # modificar
                           eps = 1e-6, kmax = 1500, 
                           precision = 6, 
                           nu = 3) # modificar
opt.contour_map(f, Xk_CGM[0], Xk_CGM[1])

### Problema: Minimizar el coste de una Máquina

Supongamos que tienes una máquina que recibe energia $E$ para realizar una tarea. Tenemos un modo que ajusta la complejidad de las operaciones de la máquina. Para tener esta máquina en funcionamiento, hay unos costes. Queremos encontrar la complejidad y la energía para minimizar los costes.

- $E$ es la Energía. (puede ser negativa!)
- $K$ es la Complejidad del sistema en unidades secretas.
- $cost(E, K)$ es el coste del trabajo.

$cost(E, K) = K^2 + E^3 + 3\cdot E \cdot K$

Encuentra los valores y decide qué algoritmo de optimización y qué parámetros proporcionan la convergencia más rápida y la solución óptima. 

> ¿Parece difícil?

Pista + Instrucciones: 
- Define la función `def f(x): return` $K^2 + E^3 + 3\cdot E \cdot K$
- Calcula la derivada y definela en `def g(x): return ...`
- Escoge un punto inicial aleatorio, con `x = np.array([np.random.rand(2)]).T * -10`.
- Juega con los parámetros `BLS_params`.
- Decide el algoritmo de optimización.
- Dibuja la direcciones de descenso.
- Dibuja la disminución de la norma del gradiente.

> Q - A
- ¿Me da infinito? Mala inicialización.
- ¿Llega al número máximo de iteraciones? Está haciendo pasitos pequeños. Intenta modificar la `rho`.

In [ ]:
# Problem
def f(x): return ...
def g(x): return ...

x = np.array([np.random.rand(2)]).T * -10

In [ ]:
BLS_params = dict(alpha_max = 2.0, 
                 alpha_min = 1e-3, # NO modificar
                 rho = 0.3, # modificar
                 c1 = 0.8, c2 = 0.4, # modificar
                 strong_wolfe = True, out = 0) # NO modificar